In [1]:
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
%%time

df = pd.read_json(r'/opt/data/articles_list_Oct12_2020.json', orient='columns')

CPU times: user 3.28 s, sys: 46.1 s, total: 49.4 s
Wall time: 49.3 s


## Basic EDA

In [10]:
df.describe()

_score
count  60374.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0

In [11]:
df.count()

_index     60374
_type      60374
_id        60374
_score     60374
_source    60374
dtype: int64

In [12]:
df.shape

(60374, 5)

In [14]:
#print 1st column 1st row
df.iloc[0] # this is same as df[[0]].values[0][0]

_index                                              articles
_type                                                   _doc
_id                                     jzHWHnUBlmHqFksVvSCK
_score                                                     1
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [4]:
df_clean= df.drop(['_id','_score', '_type'], axis=1)

In [5]:
df_clean.shape

(60374, 2)

In [18]:
df_clean.head(5)

_index                                            _source
0  articles  {'url': 'http://actionnewsjax.com/news/trendin...
1  articles  {'url': 'https://www.13newsnow.com/article/new...
2  articles  {'url': 'http://digg.com/digg-picks/link/kindl...
3  articles  {'url': 'http://sports.mynorthwest.com/1199798...
4  articles  {'url': 'https://www.aier.org/article/lockdown...

In [19]:
df_clean.iloc[0]

_index                                              articles
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [20]:
df.iloc[0]

_index                                              articles
_type                                                   _doc
_id                                     jzHWHnUBlmHqFksVvSCK
_score                                                     1
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [24]:
df.describe()

_score
count  60374.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0

In [26]:
df.dtypes

_index     object
_type      object
_id        object
_score      int64
_source    object
dtype: object

In [32]:
#Get value within a Series
df.loc[0].at['_type']

'_doc'

In [3]:
#Get value within a Series
df.loc[0].at['_source']

{'url': 'http://actionnewsjax.com/news/trending/amy-coney-barrett-hearing-day-2-questioning-begins-livestream-live-updates/MSRUGTDZVRFO3M7PXIQC6UY45U/',
 'extracted': {'title': 'Amy Coney Barrett hearing Day 2: Questioning begins; livestream, live updates',
  'date': None,
  'authors': ['Debbie Lord',
   'Cox Media Group National Content Desk',
   'Updated',
   'October',
   '- Pm',
   'Kelli Dugan',
   'Crystal Bonvillian',
   'Brianna Chambers',
   'Theresa Seiger'],
  'text': '“His judicial philosophy was straightforward: A judge must apply the law as written, not as the judge wishes it were. Sometimes that approach meant reaching results that he did not like. But as he put it in one of his best-known opinions, that is what it means to say we have a government of laws, not of men.”'},
 'metadata': {'keywords': ['amy coney barrett',
   'amy coney barrett hearing',
   'what time doe the amy coney barrett hearing start',
   'what channel is the amy coney barrett hearing on',
   'livest

In [4]:
df_src=df.drop(['_index','_id','_score', '_type'], axis=1)

In [37]:
df_src.dtypes

_source    object
dtype: object

In [5]:
%%time

df_src_norm = json_normalize(df_src['_source'])

CPU times: user 52.5 s, sys: 1min 9s, total: 2min 1s
Wall time: 2min 1s


In [ ]:
list()

In [6]:
df_src_norm.head(3)

url  \
0  http://actionnewsjax.com/news/trending/amy-con...   
1  https://www.13newsnow.com/article/news/nation-...   
2  http://digg.com/digg-picks/link/kindles-video-...   

                                     extracted.title       extracted.date  \
0  Amy Coney Barrett hearing Day 2: Questioning b...                 None   
1  Vanessa Bryant wishes Kobe, Gigi could've seen...  2020-10-12T19:24:52   
2  Kindles, Video Doorbells And More Are On Sale ...                 None   

                                   extracted.authors  \
0  [Debbie Lord, Cox Media Group National Content...   
1                     [Author, Douglas Jones, Tegna]   
2                                     [Digg Editors]   

                                      extracted.text  \
0  “His judicial philosophy was straightforward: ...   
1  Vanessa Bryant wrote that she wished Kobe and ...   
2  👋 Welcome to Digg\n\nThanks for creating an ac...   

                                   metadata.keywords  \
0  [amy coney barrett, amy coney barrett hearing,...   
1                                                 []   
2                                                 []   

                                metadata.description metadata.language  \
0  Judge Amy Coney Barrett will face questions fr...                en   
1  After the Los Angeles Lakers won the NBA champ...                en   
2  <p>If you're a Prime member, Amazon has a nice...                en   

                                        metadata.url  \
0  https://www.actionnewsjax.com/news/trending/am...   
1  https://www.13newsnow.com/article/news/nation-...   
2  https://digg.com/digg-picks/link/kindles-video...   

  metadata.dump.distribution  ...  \
0                     global  ...   
1                        NaN  ...   
2                        NaN  ...   

  metadata.dump.The Echelon exercise bike is $100 off, down to $499, right now at Walmart — and it comes with a free 6-month membership  \
0                                                NaN                                                                                      
1                                                NaN                                                                                      
2                                                NaN                                                                                      

  metadata.dump.DC.source metadata.dump.DC.date.published  \
0                     NaN                             NaN   
1                     NaN                             NaN   
2                     NaN                             NaN   

  metadata.dump.webit_document_id metadata.dump.webit_document_name  \
0                             NaN                               NaN   
1                             NaN                               NaN   
2                             NaN                               NaN   

  metadata.dump.webit_cover_date metadata.dump.burntime  \
0                            NaN                    NaN   
1                            NaN                    NaN   
2                            NaN                    NaN   

  metadata.dump.Everything we know about Ariana Grande's sixth album  \
0                                                NaN                   
1                                                NaN                   
2                                                NaN                   

  metadata.dump.article_references metadata.dump.epdf_available  
0                              NaN                          NaN  
1                              NaN                          NaN  
2                              NaN                          NaN  

[3 rows x 2481 columns]

In [41]:
df_src_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60374 entries, 0 to 60373
Columns: 2481 entries, url to metadata.dump.epdf_available
dtypes: bool(2), float64(251), object(2228)
memory usage: 1.1+ GB


In [10]:
df_src_norm.shape

(60374, 2481)

In [11]:
df_src_norm_limited=df_src_norm.filter(items=['url', 'site_name','extracted.title', 'extracted.date', 'extracted.text', 'metadata.keywords', 'metadata.description'])

In [12]:
df_src_norm_limited.shape

(60374, 6)

In [13]:
df_src_norm_limited.head(10)

url  \
0  http://actionnewsjax.com/news/trending/amy-con...   
1  https://www.13newsnow.com/article/news/nation-...   
2  http://digg.com/digg-picks/link/kindles-video-...   
3  http://sports.mynorthwest.com/1199798/clayton-...   
4  https://www.aier.org/article/lockdowns-have-ki...   
5  http://rte.ie/news/world/2020/1012/1171035-car...   
6  https://www.inforum.com/newsmd/coronavirus/671...   
7  https://www.postbulletin.com/news/crime-and-co...   
8  http://refinery29.com/de-de/2020/10/10080798/g...   
9  http://elnuevodia.com/noticias/mundo/videos/nu...   

                                     extracted.title  \
0  Amy Coney Barrett hearing Day 2: Questioning b...   
1  Vanessa Bryant wishes Kobe, Gigi could've seen...   
2  Kindles, Video Doorbells And More Are On Sale ...   
3  Seahawks Observations: John Clayton on 5-0 Haw...   
4  Lockdowns Have Killed What’s Left of the Unite...   
5  Teenager one step from becoming first millenni...   
6  Minnesota to require low-risk care homes be re...   
7  Rochester Police Department daily incident rep...   
8  „Gilmore Girls“: 16 berühmte Nebendarsteller*i...   
9  Nuevo misil balístico de Corea del Norte ¿Un d...   

                     extracted.date  \
0                              None   
1               2020-10-12T19:24:52   
2                              None   
3         2020-10-12T21:52:23+00:00   
4         2020-10-12T12:30:15+00:00   
5  2020-10-12T15:21:15.903000+01:00   
6                              None   
7                              None   
8               2020-10-10T00:00:00   
9                              None   

                                      extracted.text  \
0  “His judicial philosophy was straightforward: ...   
1  Vanessa Bryant wrote that she wished Kobe and ...   
2  👋 Welcome to Digg\n\nThanks for creating an ac...   
3  The Seahawks’ 27-26 victory over the Minnesota...   
4                                                      
5  A British-born Italian teenager who dedicated ...   
6  ROCHESTER, Minn. — Stressing the need to balan...   
7  Wondering what the commotion was in your neigh...   
8  Ich spreche hier also nicht von Melissa McCart...   
9                                                      

                                   metadata.keywords  \
0  [amy coney barrett, amy coney barrett hearing,...   
1                                                 []   
2                                                 []   
3  [MyNorthwest.com - Seattle news, sports, weath...   
4                                                 []   
5                                                 []   
6                                                 []   
7                                                 []   
8                                                 []   
9                                                 []   

                                metadata.description  
0  Judge Amy Coney Barrett will face questions fr...  
1  After the Los Angeles Lakers won the NBA champ...  
2  <p>If you're a Prime member, Amazon has a nice...  
3  What did the Vikings' D do to slow down the Se...  
4  "Long a failing institution, inaction during g...  
5  A British-born Italian teenager who dedicated ...  
6  Washington County on Wisconsin border hits 142...  
7  Wondering what the commotion was in your neigh...  
8  Achtung: Facepalm-Moment! Diese 16 „Gilmore Gi...  
9  El nuevo misil balístico intercontinental giga...

In [14]:
df_src_norm_limited['metadata.description'][0]

'Judge Amy Coney Barrett will face questions from the members of the Senate Judiciary Committee on Tuesday, as hearings into her nomination to the United States Supreme Court continue.'

In [16]:
df_src_norm_limited['extracted.text'][0]

'“His judicial philosophy was straightforward: A judge must apply the law as written, not as the judge wishes it were. Sometimes that approach meant reaching results that he did not like. But as he put it in one of his best-known opinions, that is what it means to say we have a government of laws, not of men.”'

In [55]:
df_src_norm_limited['url'][0]

'http://actionnewsjax.com/news/trending/amy-coney-barrett-hearing-day-2-questioning-begins-livestream-live-updates/MSRUGTDZVRFO3M7PXIQC6UY45U/'

In [53]:
df_src_norm_limited.describe

<bound method NDFrame.describe of                                                      url  \
0      http://actionnewsjax.com/news/trending/amy-con...   
1      https://www.13newsnow.com/article/news/nation-...   
2      http://digg.com/digg-picks/link/kindles-video-...   
3      http://sports.mynorthwest.com/1199798/clayton-...   
4      https://www.aier.org/article/lockdowns-have-ki...   
...                                                  ...   
60369  http://mediamatters.org/2020-supreme-court-nom...   
60370  https://spectrumreport.com/handmaids-tale-prot...   
60371  http://pjmedia.com/news-and-politics/rick-mora...   
60372  https://twitchy.com/samj-3930/2020/10/12/what-...   
60373  https://www.cbsnews.com/news/trump-biden-elect...   

                                         extracted.title  \
0      Amy Coney Barrett hearing Day 2: Questioning b...   
1      Vanessa Bryant wishes Kobe, Gigi could've seen...   
2      Kindles, Video Doorbells And More Are On Sale ...   
3    

# Topic Modeling

## Using Latent Dirichlet Allocation (LDA)
#### The LDA is based upon two general assumptions:

### Documents that have similar words usually have the same topic
### Documents that have groups of words frequently occurring together usually have the same topic.


# Vocabulary Creation

### Using CountVectorizer class from the sklearn.feature_extraction.text module to create a document-term matrix. We specify to only include those words that appear in less than 80% of the document and appear in at least 2 documents. We also remove all the stop words as they do not really contribute to topic modeling.

In [37]:
extracted_text=df_src_norm_limited['extracted.text']
extracted_text.dropna()

0        “His judicial philosophy was straightforward: ...
1        Vanessa Bryant wrote that she wished Kobe and ...
2        👋 Welcome to Digg\n\nThanks for creating an ac...
3        The Seahawks’ 27-26 victory over the Minnesota...
4                                                         
                               ...                        
60369    Environmental-focused news outlets such as E&E...
60370    A group of women dressed as characters from th...
60371    An internal audit conducted by the Anti-Oppres...
60372    Twitter has been especially awful today with a...
60373    Democratic presidential nominee Joe Biden is a...
Name: extracted.text, Length: 60374, dtype: object

In [38]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
#doc_term_matrix = count_vect.fit_transform(df_src_norm_limited['extracted.text'].values.astype('U'))
doc_term_matrix = count_vect.fit_transform(extracted_text.values.astype('U'))


In [39]:
doc_term_matrix

<60374x228401 sparse matrix of type '<class 'numpy.int64'>'
	with 8898350 stored elements in Compressed Sparse Row format>

In [61]:
# Has 228,401 size vocabulary in this dataset

In [40]:
%%time

# Next, we will use LDA to create topics along with the probability distribution for each word in our vocabulary for each topic. 
# LatentDirichletAllocation class from the sklearn.decomposition library to perform LDA on our document-term matrix. 
# The parameter n_components specifies the number of categories, or topics, that we want our text to be divided into. 
# The parameter random_state (aka the seed) is set to 42
# Number of categories is set by n_components

LDA = LatentDirichletAllocation(n_components=20, random_state=42)
LDA.fit(doc_term_matrix)

CPU times: user 1h 6min 13s, sys: 1min 42s, total: 1h 7min 56s
Wall time: 8min 51s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [41]:
first_topic = LDA.components_[0]

In [42]:
%%time
top_topic_words = first_topic.argsort()[-10:]

CPU times: user 22.6 ms, sys: 74 µs, total: 22.6 ms
Wall time: 19.9 ms


In [43]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

si
non
una
della
le
del
che
la
il
di


In [44]:
second_topic = LDA.components_[1]

In [45]:
%%time
top_topic_words = second_topic.argsort()[-10:]

CPU times: user 25.2 ms, sys: 139 µs, total: 25.3 ms
Wall time: 22.4 ms


In [46]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

public
time
pandemic
work
government
state
year
new
people
said


In [47]:
%%time
# 10 words with highest probabilities for all the five topics:


for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['si', 'non', 'una', 'della', 'le', 'del', 'che', 'la', 'il', 'di']


Top 10 words for topic #1:
['public', 'time', 'pandemic', 'work', 'government', 'state', 'year', 'new', 'people', 'said']


Top 10 words for topic #2:
['για', 'που', 'από', 'με', 'να', 'την', 'του', 'το', 'της', 'και']


Top 10 words for topic #3:
['en', 'billion', 'mccain', 'market', 'een', 'het', 'chinese', 'van', 'year', 'china']


Top 10 words for topic #4:
['pour', 'une', 'est', 'du', 'en', 'des', 'et', 'les', 'le', 'la']


Top 10 words for topic #5:
['para', 'por', 'del', 'se', 'las', 'los', 'en', 'que', 'el', 'la']


Top 10 words for topic #6:
['day', 'state', 'virus', 'new', 'health', 'coronavirus', 'cases', 'said', '19', 'covid']


Top 10 words for topic #7:
['win', '2020', 'year', 'time', 'games', 'league', 'team', 'said', 'season', 'game']


Top 10 words for topic #8:
['new', 'don', 'know', 'police', 'year', 'time', 'people', 'just', 'like', 'said']


Top 10 words for topic #9:
[

In [48]:
# we will add a column to the original data frame that will store the topic for the text. 
#To do so, we can use LDA.transform() method and pass it our document-term matrix. 
#This method will assign the probability of all the topics to each document

topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(60374, 20)

In [32]:
%%time
#adds a new column for topic in the data frame and assigns the topic value to each row in the column:
# To find the topic index with maximum value, we can call the argmax() method and pass 1 as the value for the axis parameter.

df_src_norm_limited['Topic'] = topic_values.argmax(axis=1)

CPU times: user 7.88 ms, sys: 3.89 ms, total: 11.8 ms
Wall time: 7.83 ms


In [33]:
df_src_norm_limited.head()

url  \
0  http://actionnewsjax.com/news/trending/amy-con...   
1  https://www.13newsnow.com/article/news/nation-...   
2  http://digg.com/digg-picks/link/kindles-video-...   
3  http://sports.mynorthwest.com/1199798/clayton-...   
4  https://www.aier.org/article/lockdowns-have-ki...   

                                     extracted.title  \
0  Amy Coney Barrett hearing Day 2: Questioning b...   
1  Vanessa Bryant wishes Kobe, Gigi could've seen...   
2  Kindles, Video Doorbells And More Are On Sale ...   
3  Seahawks Observations: John Clayton on 5-0 Haw...   
4  Lockdowns Have Killed What’s Left of the Unite...   

              extracted.date  \
0                       None   
1        2020-10-12T19:24:52   
2                       None   
3  2020-10-12T21:52:23+00:00   
4  2020-10-12T12:30:15+00:00   

                                      extracted.text  \
0  “His judicial philosophy was straightforward: ...   
1  Vanessa Bryant wrote that she wished Kobe and ...   
2  👋 Welcome to Digg\n\nThanks for creating an ac...   
3  The Seahawks’ 27-26 victory over the Minnesota...   
4                                                      

                                   metadata.keywords  \
0  [amy coney barrett, amy coney barrett hearing,...   
1                                                 []   
2                                                 []   
3  [MyNorthwest.com - Seattle news, sports, weath...   
4                                                 []   

                                metadata.description  Topic  
0  Judge Amy Coney Barrett will face questions fr...     13  
1  After the Los Angeles Lakers won the NBA champ...      7  
2  <p>If you're a Prime member, Amazon has a nice...     18  
3  What did the Vikings' D do to slow down the Se...      7  
4  "Long a failing institution, inaction during g...      0

In [17]:
df_src_norm_limited['extracted.text'][1]

'Vanessa Bryant wrote that she wished Kobe and their daughter Gianna could have been here to witness the Los Angeles Lakers winning the NBA championship.\n\nAfter the Los Angeles Lakers won their 17th NBA championship on Sunday night, Vanessa Bryant congratulated the team with an emotional message posted on social media.\n\nIn her post, the widow of the late Kobe Bryant, wrote that she wished Kobe and their daughter Gianna could have been here to witness it.\n\nKobe, along with their 13-year-old daughter Gianna "Gigi" Bryant and seven others, died in a January helicopter crash in Calabasas, California.\n\nVanessa Bryant directed a message in her Instagram story to Kobe\'s former agent and best friend, Rob Pelinka, in which she wrote, "Kobe was right, RP! \'Stay the course blockout the noise.\'\n\nShe posted a photo of Kobe and Rob together writing, "Wish Kobe and Gigi were here to see this."\n\nVanessa Bryant sends a message to Rob Pelinka and the #Lakers. pic.twitter.com/07ZXQk7wan — 

# Topic Models Using
## Non-Negative Matrix Factorization (NMF)

### Non-negative matrix factorization is also a supervised learning technique which performs clustering as well as dimensionality reduction. It can be used in combination with TF-IDF scheme to perform topic modeling.


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(extracted_text.values.astype('U'))

In [50]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [51]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

całe
6355
sonorisation
الاستفتاء
irgendeine
monolayers
damned
processo
ahmaud
quinnipiac


In [52]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [53]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

mail
state
boxes
voting
county
ballots
ballot
vote
voters
election


In [54]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['mail', 'state', 'boxes', 'voting', 'county', 'ballots', 'ballot', 'vote', 'voters', 'election']


Top 10 words for topic #1:
['coney', 'committee', 'judge', 'amy', 'justice', 'confirmation', 'senate', 'supreme', 'court', 'barrett']


Top 10 words for topic #2:
['para', 'se', 'del', 'por', 'las', 'los', 'en', 'que', 'la', 'el']


Top 10 words for topic #3:
['house', 'fauci', 'rally', 'white', 'joe', 'donald', 'campaign', 'president', 'biden', 'trump']


Top 10 words for topic #4:
['thanks', 'creating', 'continue', 'enjoy', 'card', 'articles', 'logging', 'account', 'reading', 'log']


Top 10 words for topic #5:
['ha', 'le', 'sono', 'una', 'della', 'del', 'la', 'che', 'il', 'di']


Top 10 words for topic #6:
['reported', 'deaths', 'new', 'said', 'virus', 'health', 'coronavirus', '19', 'covid', 'cases']


Top 10 words for topic #7:
['close', 'lock', 'receiving', 'notification', 'alerts', 'timely', 'icon', 'unblock', 'click', 'notifications']


Top 10 words for 

### Metadata Description

Using this field to do topic modeling.


In [55]:
metadata_description=df_src_norm_limited['metadata.description']

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(metadata_description.values.astype('U'))

In [57]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [58]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

intriguing
chaque
egyik
roller
informationen
privada
philippe
رقم
eines
fernandes


In [59]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [60]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

entertainment
business
brings
reuters
technology
politics
markets
pictures
covering
news


In [61]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['entertainment', 'business', 'brings', 'reuters', 'technology', 'politics', 'markets', 'pictures', 'covering', 'news']


Top 10 words for topic #1:
['hearings', 'senate', 'judge', 'nominee', 'confirmation', 'supreme', 'court', 'amy', 'coney', 'barrett']


Top 10 words for topic #2:
['illness', 'long', 'expect', 'tested', 'pandemic', 'health', 'positive', 'cases', '19', 'covid']


Top 10 words for topic #3:
['white', 'house', 'florida', 'joe', 'rally', 'biden', 'campaign', 'donald', 'president', 'trump']


Top 10 words for topic #4:
['عربية', 'دولية', 'الأخبار', 'والعالمي', 'جريدة', 'تهتم', 'بآخر', 'أنواعها', 'بجميع', 'المستويين']


Top 10 words for topic #5:
['authorities', 'chinese', 'country', 'tests', 'beijing', 'city', 'million', 'pandemic', 'ap', 'coronavirus']


Top 10 words for topic #6:
['se', 'il', 'una', 'le', 'que', 'del', 'di', 'el', 'en', 'la']


Top 10 words for topic #7:
['page', 'way', 'entertainment', 'affairs', 'citizen', 'sport', 'celebrit

### Keywords


In [62]:
metadata_keywords=df_src_norm_limited['metadata.keywords']

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(metadata_description.values.astype('U'))

In [64]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [65]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

lettre
privinţa
auktionsteori
mascarilla
epidemiološko
lightfoot
កទ
subscriptions
contraceptives
giovanni


In [66]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [67]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

entertainment
business
brings
reuters
technology
politics
markets
pictures
covering
news


In [68]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['entertainment', 'business', 'brings', 'reuters', 'technology', 'politics', 'markets', 'pictures', 'covering', 'news']


Top 10 words for topic #1:
['hearings', 'senate', 'judge', 'nominee', 'confirmation', 'supreme', 'court', 'amy', 'coney', 'barrett']


Top 10 words for topic #2:
['illness', 'long', 'expect', 'tested', 'pandemic', 'health', 'positive', 'cases', '19', 'covid']


Top 10 words for topic #3:
['white', 'house', 'florida', 'joe', 'rally', 'biden', 'campaign', 'donald', 'president', 'trump']


Top 10 words for topic #4:
['عربية', 'دولية', 'الأخبار', 'والعالمي', 'جريدة', 'تهتم', 'بآخر', 'أنواعها', 'بجميع', 'المستويين']


Top 10 words for topic #5:
['authorities', 'chinese', 'country', 'tests', 'beijing', 'city', 'million', 'pandemic', 'ap', 'coronavirus']


Top 10 words for topic #6:
['se', 'il', 'una', 'le', 'que', 'del', 'di', 'el', 'en', 'la']


Top 10 words for topic #7:
['page', 'way', 'entertainment', 'affairs', 'citizen', 'sport', 'celebrit